In [19]:
import keras
import csv
import scipy
import numpy as np
import nltk #from nltk.tokenize import punkt #.tokenize.moses import MosesTokenizer
from gensim.models import KeyedVectors
from itertools import islice
import pickle
import pandas as pd
import re
from nltk.corpus import stopwords
import json
import pickle
from cyordereddict import OrderedDict
from joblib import Parallel, delayed

DATASET_FILE_PATH = "./DataSet/yelp.csv/yelp.csv"
WORD2VEC_FILE_PATH = "./DataSet/FBword2vec/wiki.en.vec"
TRAINING_DATA_PATH = "./DataSet/trainingData"
EXTENDED_DATASET_PATH = "./DataSet/review.json"

###File Names
TEST_NAME = "SPACY_tokenizer_No_Stopwords_noPunctuation_FB_vectors_100percentDataset"

EMBEDDED_MATRIX_NAME = TEST_NAME+'_embedded_matrix.pkl'
TOKENIZED_SENTENCES_NAME = TEST_NAME+ "_tokenized_sentences.pkl"
WORD_INDEX_NAME = TEST_NAME+ "_word_index.pkl"
INDEXED_TOKENIZED_SENTENCES_NAME = TEST_NAME + "_indexed_tokenized_sentences.pkl"

LABELS_NAME = 'labels.pkl'


AMOUNT_OF_WORDS = 70
FACEBOOK_EMBEDDING_SIZE = 300

In [20]:
word_index = pickle.load(open(WORD_INDEX_NAME,'rb'))

In [21]:
embedded_matrix =  np.load(EMBEDDED_MATRIX_NAME+'.npy')
print('embedded matrix loaded')

embedded matrix loaded


In [22]:
indexed_tokenized_sentences = pickle.load(open(INDEXED_TOKENIZED_SENTENCES_NAME,'rb'))

In [23]:
def reshapeSentences(indexed_tokenized_sentences,sentence_length):
    for i in range(len(indexed_tokenized_sentences)):
        padding = sentence_length - len(indexed_tokenized_sentences[i])
        indexed_tokenized_sentences[i] = indexed_tokenized_sentences[i][:sentence_length] + [-1]*padding
    return indexed_tokenized_sentences

In [24]:
### BUILDING MODEL

In [25]:
reshaped_Sentences = reshapeSentences(indexed_tokenized_sentences, AMOUNT_OF_WORDS)

In [26]:

labels = pd.read_pickle(LABELS_NAME)
labels.head()
labels.pop('cool')
labels.pop('funny')
labels.pop('useful')


0           0
1           0
2           0
3           0
4           0
5           0
6           1
7           0
8           1
9           1
10          3
11          1
12          5
13          9
14          2
15          2
16          0
17          0
18          2
19          4
20          9
21          1
22          0
23          0
24          4
25          0
26          1
27          0
28          1
29          0
           ..
5261639     0
5261640     1
5261641     0
5261642     0
5261643     0
5261644     1
5261645     2
5261646     1
5261647     0
5261648     7
5261649     0
5261650     0
5261651     0
5261652     1
5261653     0
5261654     1
5261655     5
5261656     1
5261657     1
5261658     5
5261659     1
5261660     2
5261661     0
5261662     4
5261663     0
5261664    36
5261665     0
5261666     3
5261667     2
5261668     0
Name: useful, Length: 5261669, dtype: int64

In [27]:
###Class Balancing: reduces accuracy by 5%
labels['text'] = reshaped_Sentences
max_retrieve = len(labels.index)
class_balancing = labels.groupby('stars')
class_balancing = class_balancing.apply(lambda x: x.sample(class_balancing.size().min()).reset_index(drop=True))
labels = class_balancing.head(n=max_retrieve)
reshaped_Sentences = labels.pop('text').values

lengths = []
for i in range(1,6):
    s = labels.stars
    s = s[s == i]
    lengths.append(len(s))
print(lengths)

[438161, 438161, 438161, 438161, 438161]


In [28]:
labels = pd.get_dummies(labels['stars'])
labels.head()

1  2  3  4  5
stars                 
1     0  1  0  0  0  0
      1  1  0  0  0  0
      2  1  0  0  0  0
      3  1  0  0  0  0
      4  1  0  0  0  0

In [29]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import numpy as np
from keras import backend
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [30]:
embedding_layer = Embedding(
    len(word_index),
    FACEBOOK_EMBEDDING_SIZE,
    weights=[embedded_matrix],
    input_length=AMOUNT_OF_WORDS,
    trainable=True
)

In [31]:
reshaped_Sentences = np.stack(reshaped_Sentences,axis=0)

In [32]:

X_train, X_test, y_train, y_test = train_test_split(reshaped_Sentences, labels.values, test_size=0.2, random_state=42)

In [33]:
def generator(features, labels, batch_size, replace=True):
    
    batch_features = np.zeros((batch_size,AMOUNT_OF_WORDS))
    batch_labels = np.zeros((batch_size,5))
    while True:
        index = np.random.choice(features.shape[0], batch_size, replace=replace)
        batch_features[:] = features[index]
        batch_labels[:] = labels[index]
        yield batch_features, batch_labels
        

In [34]:
def percentageCorrect(y_true, y_pred):
    return backend.mean(backend.equal(y_true, backend.round(y_pred)))*100

In [35]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.25))
model.add(Conv1D(32, 5, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=5))
model.add(LSTM(70))
model.add(Dense(200))
model.add(Dropout(0.8))
model.add(Dense(200))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 70, 300)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 70, 300)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 66, 32)            48032     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 70)                28840     
_________________________________________________________________
dense_4 (Dense)              (None, 200)               14200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
__________

In [36]:
#model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=40, batch_size=10, verbose=2)
model.fit_generator(generator(X_train,y_train,7000,False),steps_per_epoch=100, epochs=30, verbose=2, validation_data=generator(X_test, y_test, 20000, False), validation_steps=5)

AttributeError: 'Tensor' object has no attribute 'assign'

In [42]:
X_test[2]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1])